#### Train test split, Cross Val score, Hyper Parameter Tuning

In [83]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler


In [84]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier

In [85]:

file_path = 'data3.csv'
df = pd.read_csv(file_path)

In [86]:
df.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1,1
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2,1
2,41,0,1,130,204,0,0,172,0,1.4,2,0,2,1
3,56,1,1,120,236,0,1,178,0,0.8,2,0,2,1
4,57,0,0,120,354,0,1,163,1,0.6,2,0,2,1


In [87]:
df.shape

(303, 14)

In [88]:
df.isnull().sum()

age         0
sex         0
cp          0
trestbps    0
chol        0
fbs         0
restecg     0
thalach     0
exang       0
oldpeak     0
slope       0
ca          0
thal        0
target      0
dtype: int64

In [89]:
df['target'].value_counts()

target
1    165
0    138
Name: count, dtype: int64

## note

- label 1 = defective heart
- label 2 = healthy heart

In [90]:
X = df.drop(columns='target', axis = 1)
Y = df['target']

In [91]:
X

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2
2,41,0,1,130,204,0,0,172,0,1.4,2,0,2
3,56,1,1,120,236,0,1,178,0,0.8,2,0,2
4,57,0,0,120,354,0,1,163,1,0.6,2,0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...
298,57,0,0,140,241,0,1,123,1,0.2,1,0,3
299,45,1,3,110,264,0,1,132,0,1.2,1,0,3
300,68,1,0,144,193,1,1,141,0,3.4,1,2,3
301,57,1,0,130,131,0,1,115,1,1.2,1,1,3


In [92]:
Y

0      1
1      1
2      1
3      1
4      1
      ..
298    0
299    0
300    0
301    0
302    0
Name: target, Length: 303, dtype: int64

#### Train test Split

In [93]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, stratify = Y, random_state= 3)

In [94]:
print(X.shape, X_train.shape,X_test.shape)

(303, 13) (242, 13) (61, 13)


- Comparing the performance of models

In [123]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)
models = [LogisticRegression(max_iter=2000, solver='liblinear'), SVC(kernel='linear'), KNeighborsClassifier(), RandomForestClassifier()]


In [124]:
def compare_models_train_test():
    for model in models:

        model.fit(X_train, Y_train)

        test_data_prediction = model.predict(X_test)

        accuracy = accuracy_score(Y_test, test_data_prediction)

        print('accuracy score', model, ' : ', accuracy)

In [125]:
compare_models_train_test()

accuracy score LogisticRegression(max_iter=2000, solver='liblinear')  :  0.7868852459016393
accuracy score SVC(kernel='linear')  :  0.7704918032786885
accuracy score KNeighborsClassifier()  :  0.8032786885245902
accuracy score RandomForestClassifier()  :  0.7704918032786885


#### Cross Val Score

In [126]:
cv_score_lr = cross_val_score(LogisticRegression(max_iter=2000), X, Y, cv=5)
print('Cross-validation scores for Logistic Regression:', cv_score_lr)
mean_accuracy = print('Mean cross-validation score for Logistic Regression:', np.mean(cv_score_lr))


Cross-validation scores for Logistic Regression: [0.80327869 0.86885246 0.85245902 0.86666667 0.75      ]
Mean cross-validation score for Logistic Regression: 0.8282513661202187


In [127]:
cv_score_df = pd.DataFrame({'Cross-validation scores': cv_score_lr})

cv_score_df.loc['Mean'] = np.mean(cv_score_lr)
print(cv_score_df)


      Cross-validation scores
0                    0.803279
1                    0.868852
2                    0.852459
3                    0.866667
4                    0.750000
Mean                 0.828251


In [128]:
def cross_validation():
    for model in models:
        cv_score = cross_val_score(model, X, Y, cv=5)
        mean = np.mean(cv_score)
        mean_accuracy = round(mean, 2)
        print("CV score for", model, ':', mean_accuracy)

cross_validation()

CV score for LogisticRegression(max_iter=2000, solver='liblinear') : 0.82
CV score for SVC(kernel='linear') : 0.83
CV score for KNeighborsClassifier() : 0.64
CV score for RandomForestClassifier() : 0.83
